<a href="https://colab.research.google.com/github/afnanhabib787/Artificial-Intelligence/blob/main/CNN_TRANSFER_LEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hi and Welcome!
## First thing you need to upload the dataset (zip file) to your google drive I provided, because I have arranged the test, training and validation folders manually
## Then you need connect your google drive with the following code

In [ ]:
"""
@author: Afnan Habib aka AI__Guru
"""

google_colab_flag = True # Make it False if do not intend to use Google Colab and want to train in local machine!!

# For training in Google Colab
if(google_colab_flag):
  from google.colab import drive
  drive.mount('/content/drive')
  !ls
  import sys
  root_path = '/content/' # This is the path to where in google drive the code is stored!
  sys.path.append(root_path)

# For local training
else:
  root_path = ''

Mounted at /content/drive
drive  sample_data


## This code will unzip the dataset file to your colab environment

In [ ]:
!unzip /content/drive/MyDrive/DS2.zip

Archive:  /content/drive/MyDrive/DS2.zip
   creating: DS2/
   creating: DS2/test/
   creating: DS2/test/adas/
   creating: DS2/test/jahe/
   creating: DS2/test/jintan/
   creating: DS2/test/kencur/
   creating: DS2/test/ketumbar/
   creating: DS2/test/kunci/
   creating: DS2/test/kunyit/
   creating: DS2/test/laos/
   creating: DS2/test/merica/
   creating: DS2/test/temulawak/
   creating: DS2/train/
   creating: DS2/train/adas/
  inflating: DS2/train/adas/adas_100.jpg  
  inflating: DS2/train/adas/adas_26.jpg  
  inflating: DS2/train/adas/adas_27.jpg  
  inflating: DS2/train/adas/adas_28.jpg  
  inflating: DS2/train/adas/adas_29.jpg  
  inflating: DS2/train/adas/adas_30.jpg  
  inflating: DS2/train/adas/adas_31.jpg  
  inflating: DS2/train/adas/adas_32.jpg  
  inflating: DS2/train/adas/adas_33.jpg  
  inflating: DS2/train/adas/adas_34.jpg  
  inflating: DS2/train/adas/adas_35.jpg  
  inflating: DS2/train/adas/adas_36.jpg  
  inflating: DS2/train/adas/adas_37.jpg  
  inflating: DS2/tra

# Add all necessary libraries!

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
datasetFolderName=root_path+'DS2'
MODEL_FILENAME=root_path+"model_cv.h5"
sourceFiles=[]
classLabels=['adas', 'jahe', 'jintan','kencur', 'ketumbar','kunci','kunyit','laos','merica','temulawak']
X=[]
Y=[]

img_rows, img_cols =  150, 150 # input image dimensions
train_path=datasetFolderName+'/train/'
validation_path=datasetFolderName+'/validation/'
test_path=datasetFolderName+'/test/'

## Folowing code will transfer files between training and test folder and we will use this test folder later to evaluate the model

In [ ]:
def transferBetweenFolders(source, dest, splitRate): 
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
            print(source+str(sourceFiles[eachIndex]))
            print(dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")
        
def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', 
                               splitRate)

def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score


# First, check if test folder is empty or not, if not transfer all existing files to train.

In [ ]:
transferAllClassBetweenFolders('test', 'train', 1.0)

No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!


# Now, split some part of train data into the test folders.

In [ ]:
transferAllClassBetweenFolders('train', 'test', 0.20)

/content/DS2/train/adas/adas_100.jpg
/content/DS2/test/adas/adas_100.jpg
/content/DS2/train/adas/adas_99.jpg
/content/DS2/test/adas/adas_99.jpg
/content/DS2/train/adas/adas_77.jpg
/content/DS2/test/adas/adas_77.jpg
/content/DS2/train/adas/adas_96.jpg
/content/DS2/test/adas/adas_96.jpg
/content/DS2/train/adas/adas_28.jpg
/content/DS2/test/adas/adas_28.jpg
/content/DS2/train/adas/adas_92.jpg
/content/DS2/test/adas/adas_92.jpg
/content/DS2/train/adas/adas_39.jpg
/content/DS2/test/adas/adas_39.jpg
/content/DS2/train/adas/adas_33.jpg
/content/DS2/test/adas/adas_33.jpg
/content/DS2/train/adas/adas_81.jpg
/content/DS2/test/adas/adas_81.jpg
/content/DS2/train/adas/adas_97.jpg
/content/DS2/test/adas/adas_97.jpg
/content/DS2/train/adas/adas_50.jpg
/content/DS2/test/adas/adas_50.jpg
/content/DS2/train/adas/adas_52.jpg
/content/DS2/test/adas/adas_52.jpg
/content/DS2/train/adas/adas_89.jpg
/content/DS2/test/adas/adas_89.jpg
/content/DS2/train/adas/adas_87.jpg
/content/DS2/test/adas/adas_87.jpg
/con

## Following code will make two lists X and Y, X contains the names of the files and Y contains the class number

In [ ]:
def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(classLabels)):
          if(folderName==classLabels[i]):
              Y.append(i)

###  Organize file names and class labels in X and Y variables

In [ ]:
for i in range(len(classLabels)):
  prepareNameWithLabels(classLabels[i])

In [ ]:
X=np.asarray(X)
Y=np.asarray(Y)

In [ ]:
print(X)
print(Y)


['adas_91.jpg' 'adas_40.jpg' 'adas_61.jpg' 'adas_26.jpg' 'adas_94.jpg'
 'adas_30.jpg' 'adas_54.jpg' 'adas_70.jpg' 'adas_76.jpg' 'adas_78.jpg'
 'adas_65.jpg' 'adas_72.jpg' 'adas_48.jpg' 'adas_36.jpg' 'adas_95.jpg'
 'adas_49.jpg' 'adas_84.jpg' 'adas_46.jpg' 'adas_71.jpg' 'adas_55.jpg'
 'adas_86.jpg' 'adas_34.jpg' 'adas_53.jpg' 'adas_43.jpg' 'adas_27.jpg'
 'adas_41.jpg' 'adas_67.jpg' 'adas_66.jpg' 'adas_90.jpg' 'adas_58.jpg'
 'adas_37.jpg' 'adas_85.jpg' 'adas_56.jpg' 'adas_68.jpg' 'adas_29.jpg'
 'adas_32.jpg' 'adas_83.jpg' 'adas_38.jpg' 'adas_51.jpg' 'adas_69.jpg'
 'adas_93.jpg' 'adas_88.jpg' 'adas_45.jpg' 'adas_75.jpg' 'adas_35.jpg'
 'adas_82.jpg' 'adas_60.jpg' 'adas_59.jpg' 'adas_47.jpg' 'adas_79.jpg'
 'adas_80.jpg' 'adas_31.jpg' 'adas_98.jpg' 'adas_63.jpg' 'adas_74.jpg'
 'adas_73.jpg' 'adas_57.jpg' 'adas_62.jpg' 'adas_44.jpg' 'adas_64.jpg'
 'jahe_100.jpg' 'jahe_72.jpg' 'jahe_87.jpg' 'jahe_38.jpg' 'jahe_64.jpg'
 'jahe_30.jpg' 'jahe_43.jpg' 'jahe_42.jpg' 'jahe_60.jpg' 'jahe_80.jpg'
 'jah

## Import VGG16 pretrained model for transfer learning

In [ ]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
include_top=False,
input_shape=(150, 150, 3))

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
len(classLabels)

10

## Here I concatenated my custom layers on top of VGG16 model with 10 output lables


In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
batch_size = 100
epoch=10
activationFunction='elu'
def getModel():
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(Dense(len(classLabels), activation='softmax')) 
    
    conv_base.trainable = False

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model=getModel()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 16,814,666
Trainable params: 2,099,978
Non-trainable params: 14,714,688
_________________________________________________________________


# **Stratified K-Fold Cross validation**

In [ ]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        for i in range(len(classLabels)):
          if(Y_val[eachIndex]==i):
              classLabel=classLabels[i]
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
        
    train_datagen = ImageDataGenerator(
                      rescale=1./255,
                      zoom_range=0.20,
                      fill_mode="nearest",
                      rotation_range=40,
                      width_shift_range=0.2,
                      height_shift_range=0.2,
                      shear_range=0.2,
                      horizontal_flip=True,
                      )
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
        
    #Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)   
   
    # fit model
    history=model.fit_generator(train_generator, 
                        epochs=10)
    
    predictions = model.predict_generator(validation_generator, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")    
    valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)

/content/DS2/validation/adas/adas_63.jpg
/content/DS2/train/adas/adas_63.jpg
/content/DS2/validation/adas/adas_84.jpg
/content/DS2/train/adas/adas_84.jpg
/content/DS2/validation/adas/adas_29.jpg
/content/DS2/train/adas/adas_29.jpg
/content/DS2/validation/adas/adas_59.jpg
/content/DS2/train/adas/adas_59.jpg
/content/DS2/validation/adas/adas_80.jpg
/content/DS2/train/adas/adas_80.jpg
/content/DS2/validation/adas/adas_85.jpg
/content/DS2/train/adas/adas_85.jpg
/content/DS2/validation/adas/adas_38.jpg
/content/DS2/train/adas/adas_38.jpg
/content/DS2/validation/adas/adas_41.jpg
/content/DS2/train/adas/adas_41.jpg
/content/DS2/validation/adas/adas_95.jpg
/content/DS2/train/adas/adas_95.jpg
/content/DS2/validation/adas/adas_43.jpg
/content/DS2/train/adas/adas_43.jpg
/content/DS2/validation/adas/adas_73.jpg
/content/DS2/train/adas/adas_73.jpg
/content/DS2/validation/adas/adas_48.jpg
/content/DS2/train/adas/adas_48.jpg
/content/DS2/validation/adas/adas_98.jpg
/content/DS2/train/adas/adas_98.jpg

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
4/4 [==============================] - 70s 22s/step - loss: 0.3448 - accuracy: 0.8836
Epoch 2/10
4/4 [==============================] - 71s 16s/step - loss: 0.2425 - accuracy: 0.9497
Epoch 3/10
4/4 [==============================] - 73s 23s/step - loss: 0.2583 - accuracy: 0.9025
Epoch 4/10
4/4 [==============================] - 72s 16s/step - loss: 0.2729 - accuracy: 0.9182
Epoch 5/10
4/4 [==============================] - 77s 24s/step - loss: 0.1951 - accuracy: 0.9497
Epoch 6/10
4/4 [==============================] - 69s 16s/step - loss: 0.2187 - accuracy: 0.9465
Epoch 7/10
4/4 [==============================] - 68s 16s/step - loss: 0.1690 - accuracy: 0.9528
Epoch 8/10
4/4 [==============================] - 68s 15s/step - loss: 0.2099 - accuracy: 0.9403
Epoch 9/10
4/4 [==============================] - 68s 16s/step - loss: 0.2372 - accuracy: 0.9119
Epoch 10/10
4/4 [==============================] - 68s 15s/step - loss: 0.2163 - accuracy: 0.9340


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


2/2 [==============================] - 33s 12s/step
***Performance on Validation data***
Accuracy  : 0.9811320754716981
Precision : 0.9846698113207547
f1Score : 0.9817944331065128
[[20  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0]
 [ 0  0  0 13  0  0  0  0  0  1]
 [ 0  0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0]
 [ 0  0  0  0  0  0 13  0  0  1]
 [ 0  0  0  0  0  0  0 19  0  1]
 [ 0  0  0  0  0  0  0  0 13  0]
 [ 0  0  0  0  0  0  0  0  0 13]]
/content/DS2/validation/adas/adas_63.jpg
/content/DS2/train/adas/adas_63.jpg
/content/DS2/validation/adas/adas_88.jpg
/content/DS2/train/adas/adas_88.jpg
/content/DS2/validation/adas/adas_27.jpg
/content/DS2/train/adas/adas_27.jpg
/content/DS2/validation/adas/adas_72.jpg
/content/DS2/train/adas/adas_72.jpg
/content/DS2/validation/adas/adas_26.jpg
/content/DS2/train/adas/adas_26.jpg
/content/DS2/validation/adas/adas_56.jpg
/content/DS2/train/adas/adas_56.jpg
/content/DS2/validation/ada

...

2/2 [==============================] - 33s 12s/step
***Performance on Validation data***
Accuracy  : 1.0
Precision : 1.0
f1Score : 1.0
[[20  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0]
 [ 0  0  0 14  0  0  0  0  0  0]
 [ 0  0  0  0 13  0  0  0  0  0]
 [ 0  0  0  0  0 14  0  0  0  0]
 [ 0  0  0  0  0  0 13  0  0  0]
 [ 0  0  0  0  0  0  0 20  0  0]
 [ 0  0  0  0  0  0  0  0 14  0]
 [ 0  0  0  0  0  0  0  0  0 13]]
/content/DS2/validation/adas/adas_45.jpg
/content/DS2/train/adas/adas_45.jpg
/content/DS2/validation/adas/adas_29.jpg
/content/DS2/train/adas/adas_29.jpg
/content/DS2/validation/adas/adas_48.jpg
/content/DS2/train/adas/adas_48.jpg
/content/DS2/validation/adas/adas_62.jpg
/content/DS2/train/adas/adas_62.jpg
/content/DS2/validation/adas/adas_59.jpg
/content/DS2/train/adas/adas_59.jpg
/content/DS2/validation/adas/adas_67.jpg
/content/DS2/train/adas/adas_67.jpg
/content/DS2/validation/adas/adas_55.jpg
/content/DS2/train/adas/adas_55

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
4/4 [==============================] - 68s 16s/step - loss: 0.1801 - accuracy: 0.9528
Epoch 2/10
4/4 [==============================] - 65s 20s/step - loss: 0.1407 - accuracy: 0.9560
Epoch 3/10
4/4 [==============================] - 66s 15s/step - loss: 0.1154 - accuracy: 0.9717
Epoch 4/10
4/4 [==============================] - 67s 15s/step - loss: 0.1541 - accuracy: 0.9560
Epoch 5/10
4/4 [==============================] - 67s 15s/step - loss: 0.1145 - accuracy: 0.9748
Epoch 6/10
4/4 [==============================] - 65s 21s/step - loss: 0.1296 - accuracy: 0.9591
Epoch 7/10
4/4 [==============================] - 65s 20s/step - loss: 0.1390 - accuracy: 0.9528
Epoch 8/10
4/4 [==============================] - 67s 15s/step - loss: 0.0991 - accuracy: 0.9780
Epoch 9/10
4/4 [==============================] - 67s 21s/step - loss: 0.1036 - accuracy: 0.9811
Epoch 10/10
4/4 [==============================] - 65s 15s/step - loss: 0.1099 - accuracy: 0.9686


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


2/2 [==============================] - 31s 12s/step
***Performance on Validation data***
Accuracy  : 0.9874213836477987
Precision : 0.9886792452830188
f1Score : 0.9872686069309705
[[20  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0]
 [ 0  0  0 14  0  0  0  0  0  0]
 [ 0  0  0  0 13  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0]
 [ 0  0  0  0  0  0 14  0  0  0]
 [ 0  0  0  0  0  0  0 20  0  0]
 [ 0  0  0  0  0  0  0  0 13  0]
 [ 0  2  0  0  0  0  0  0  0 12]]


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 16,814,666
Trainable params: 2,099,978
Non-trainable params: 14,714,688
_________________________________________________________________


# Test phase

In [ ]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) 
predictions = model.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore = my_metrics(true_classes, yPredictions)
model.save(MODEL_FILENAME)

==============TEST RESULTS============
Found 118 images belonging to 10 classes.
2/2 [==============================] - 25s 4s/step
Accuracy  : 0.9830508474576272
Precision : 0.9838212634822805
f1Score : 0.9830296079181005
[[15  0  0  0  0  0  0  0  0  0]
 [ 0 13  0  0  0  0  0  0  0  0]
 [ 0  0 15  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  1  0  9  0  0  0  0]
 [ 0  0  0  0  0  1  9  0  0  0]
 [ 0  0  0  0  0  0  0 15  0  0]
 [ 0  0  0  0  0  0  0  0 10  0]
 [ 0  0  0  0  0  0  0  0  0 10]]


## Variation 2

## Added MaxPooling2D layer after VGG16 model

In [ ]:
def getModel2():
    model2 = models.Sequential()
    model2.add(conv_base)
    model2.add(layers.MaxPooling2D((2, 2)))
    model2.add(layers.Flatten())
    model2.add(layers.Dense(256, activation='relu'))
    model2.add(Dense(len(classLabels), activation='softmax')) 
    
    conv_base.trainable = False

    model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model2

model2=getModel2()

In [ ]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 512)        0         
 )                                                               
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dense_5 (Dense)             (None, 10)                2570      
                                                                 
Total params: 15,241,802
Trainable params: 527,114
Non-trainable params: 14,714,688
____________________________________

In [ ]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        for i in range(len(classLabels)):
          if(Y_val[eachIndex]==i):
              classLabel=classLabels[i]
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
        
    train_datagen = ImageDataGenerator(
                      rescale=1./255,
                      zoom_range=0.20,
                      fill_mode="nearest",
                      rotation_range=40,
                      width_shift_range=0.2,
                      height_shift_range=0.2,
                      shear_range=0.2,
                      horizontal_flip=True,
                      )
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
        
    #Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)   
   
    # fit model
    history=model2.fit_generator(train_generator, 
                        epochs=10)
    
    predictions = model2.predict_generator(validation_generator, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")    
    valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)

/content/DS2/validation/adas/adas_86.jpg
/content/DS2/train/adas/adas_86.jpg
/content/DS2/validation/adas/adas_46.jpg
/content/DS2/train/adas/adas_46.jpg
/content/DS2/validation/adas/adas_98.jpg
/content/DS2/train/adas/adas_98.jpg
/content/DS2/validation/adas/adas_64.jpg
/content/DS2/train/adas/adas_64.jpg
/content/DS2/validation/adas/adas_62.jpg
/content/DS2/train/adas/adas_62.jpg
/content/DS2/validation/adas/adas_71.jpg
/content/DS2/train/adas/adas_71.jpg
/content/DS2/validation/adas/adas_54.jpg
/content/DS2/train/adas/adas_54.jpg
/content/DS2/validation/adas/adas_29.jpg
/content/DS2/train/adas/adas_29.jpg
/content/DS2/validation/adas/adas_44.jpg
/content/DS2/train/adas/adas_44.jpg
/content/DS2/validation/adas/adas_70.jpg
/content/DS2/train/adas/adas_70.jpg
/content/DS2/validation/adas/adas_59.jpg
/content/DS2/train/adas/adas_59.jpg
/content/DS2/validation/adas/adas_35.jpg
/content/DS2/train/adas/adas_35.jpg
/content/DS2/validation/adas/adas_57.jpg
/content/DS2/train/adas/adas_57.jpg

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
4/4 [==============================] - 65s 15s/step - loss: 2.4106 - accuracy: 0.2516
Epoch 2/10
4/4 [==============================] - 65s 15s/step - loss: 1.3814 - accuracy: 0.5629
Epoch 3/10
4/4 [==============================] - 64s 20s/step - loss: 1.1149 - accuracy: 0.5943
Epoch 4/10
4/4 [==============================] - 65s 20s/step - loss: 0.9793 - accuracy: 0.6635
Epoch 5/10
4/4 [==============================] - 64s 15s/step - loss: 0.7480 - accuracy: 0.7421
Epoch 6/10
4/4 [==============================] - 63s 20s/step - loss: 0.7105 - accuracy: 0.7327
Epoch 7/10
4/4 [==============================] - 64s 15s/step - loss: 0.5906 - accuracy: 0.8333
Epoch 8/10
4/4 [==============================] - 64s 14s/step - loss: 0.4925 - accuracy: 0.8679
Epoch 9/10
4/4 [==============================] - 65s 15s/step - loss: 0.5001 - accuracy: 0.8522
Epoch 10/10
4/4 [==============================] - 66s 15s/step - loss: 0.4068 - accuracy: 0.8899


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


2/2 [==============================] - 32s 12s/step
***Performance on Validation data***
Accuracy  : 0.8553459119496856
Precision : 0.8656064323517153
f1Score : 0.8539198140102624
[[20  0  0  0  0  0  0  0  0  0]
 [ 0 13  0  2  0  0  2  0  0  1]
 [ 0  0 20  0  0  0  0  0  0  0]
 [ 0  1  0  8  0  2  0  0  0  3]
 [ 0  0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0  0 12  0  1  0  0]
 [ 0  0  0  0  0  0  9  0  0  5]
 [ 0  0  0  0  0  0  0 20  0  0]
 [ 0  0  0  0  0  0  0  0 13  0]
 [ 1  0  0  0  0  0  0  5  0  7]]
/content/DS2/validation/adas/adas_69.jpg
/content/DS2/train/adas/adas_69.jpg
/content/DS2/validation/adas/adas_71.jpg
/content/DS2/train/adas/adas_71.jpg
/content/DS2/validation/adas/adas_53.jpg
/content/DS2/train/adas/adas_53.jpg
/content/DS2/validation/adas/adas_30.jpg
/content/DS2/train/adas/adas_30.jpg
/content/DS2/validation/adas/adas_85.jpg
/content/DS2/train/adas/adas_85.jpg
/content/DS2/validation/adas/adas_98.jpg
/content/DS2/train/adas/adas_98.jpg
/content/DS2/validation/ada

...

2/2 [==============================] - 31s 12s/step
***Performance on Validation data***
Accuracy  : 0.9685534591194969
Precision : 0.9712488769092543
f1Score : 0.968920425021054
[[20  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0]
 [ 0  0  0 13  0  0  0  0  0  1]
 [ 0  0  0  0 13  0  0  0  0  0]
 [ 0  1  0  0  0 13  0  0  0  0]
 [ 0  0  0  0  0  0 12  0  0  1]
 [ 0  0  0  0  0  0  0 20  0  0]
 [ 0  0  0  0  0  0  0  0 14  0]
 [ 0  2  0  0  0  0  0  0  0 11]]
/content/DS2/validation/adas/adas_41.jpg
/content/DS2/train/adas/adas_41.jpg
/content/DS2/validation/adas/adas_61.jpg
/content/DS2/train/adas/adas_61.jpg
/content/DS2/validation/adas/adas_80.jpg
/content/DS2/train/adas/adas_80.jpg
/content/DS2/validation/adas/adas_73.jpg
/content/DS2/train/adas/adas_73.jpg
/content/DS2/validation/adas/adas_40.jpg
/content/DS2/train/adas/adas_40.jpg
/content/DS2/validation/adas/adas_88.jpg
/content/DS2/train/adas/adas_88.jpg
/content/DS2/validation/adas

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
4/4 [==============================] - 68s 21s/step - loss: 0.2802 - accuracy: 0.9340
Epoch 2/10
4/4 [==============================] - 66s 21s/step - loss: 0.3159 - accuracy: 0.9214
Epoch 3/10
4/4 [==============================] - 65s 15s/step - loss: 0.2814 - accuracy: 0.9151
Epoch 4/10
4/4 [==============================] - 66s 15s/step - loss: 0.2292 - accuracy: 0.9214
Epoch 5/10
4/4 [==============================] - 68s 16s/step - loss: 0.2752 - accuracy: 0.9025
Epoch 6/10
4/4 [==============================] - 65s 15s/step - loss: 0.2268 - accuracy: 0.9434
Epoch 7/10
4/4 [==============================] - 65s 20s/step - loss: 0.2458 - accuracy: 0.9151
Epoch 8/10
4/4 [==============================] - 67s 15s/step - loss: 0.1940 - accuracy: 0.9465
Epoch 9/10
4/4 [==============================] - 69s 16s/step - loss: 0.1906 - accuracy: 0.9434
Epoch 10/10
4/4 [==============================] - 68s 15s/step - loss: 0.1792 - accuracy: 0.9528


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


2/2 [==============================] - 32s 12s/step
***Performance on Validation data***
Accuracy  : 0.9937106918238994
Precision : 0.9940417080436942
f1Score : 0.993679213805
[[20  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0]
 [ 0  0  0 14  0  0  0  0  0  0]
 [ 0  0  0  0 13  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0]
 [ 0  0  0  0  0  0 14  0  0  0]
 [ 0  0  0  0  0  0  0 20  0  0]
 [ 0  0  0  0  0  0  0  0 13  0]
 [ 0  1  0  0  0  0  0  0  0 13]]


In [ ]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) 
predictions = model2.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore = my_metrics(true_classes, yPredictions)
model.save(MODEL_FILENAME)

==============TEST RESULTS============
Found 118 images belonging to 10 classes.
2/2 [==============================] - 24s 4s/step
Accuracy  : 0.9830508474576272
Precision : 0.9845916795069338
f1Score : 0.9830852701596188
[[15  0  0  0  0  0  0  0  0  0]
 [ 0 13  0  0  0  0  0  0  0  0]
 [ 0  0 15  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0  0]
 [ 0  0  0  0  0  1  9  0  0  0]
 [ 0  0  0  0  0  0  0 14  0  1]
 [ 0  0  0  0  0  0  0  0 10  0]
 [ 0  0  0  0  0  0  0  0  0 10]]


## Variation 3

## Added another MaxPooling layer, changed optimizer to SGD, number of neurons in Dense layer changed to 128.

In [ ]:
def getModel3():
    model3 = models.Sequential()
    model3.add(conv_base)
    model3.add(layers.MaxPooling2D((2, 2)))
    model3.add(layers.MaxPooling2D((2, 2)))
    model3.add(layers.Flatten())
    model3.add(layers.Dense(128, activation='relu'))
    model3.add(Dense(len(classLabels), activation='softmax')) 
    
    conv_base.trainable = False

    model3.compile(optimizer= 'SGD' , loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model3

model3=getModel3()

In [ ]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        for i in range(len(classLabels)):
          if(Y_val[eachIndex]==i):
              classLabel=classLabels[i]
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
        
    train_datagen = ImageDataGenerator(
                      rescale=1./255,
                      zoom_range=0.20,
                      fill_mode="nearest",
                      rotation_range=40,
                      width_shift_range=0.2,
                      height_shift_range=0.2,
                      shear_range=0.2,
                      horizontal_flip=True,
                      )
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
        
    #Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)   
   
    # fit model
    history=model3.fit_generator(train_generator, 
                        epochs=10)
    
    predictions = model3.predict_generator(validation_generator, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")    
    valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)

/content/DS2/validation/adas/adas_32.jpg
/content/DS2/train/adas/adas_32.jpg
/content/DS2/validation/adas/adas_82.jpg
/content/DS2/train/adas/adas_82.jpg
/content/DS2/validation/adas/adas_79.jpg
/content/DS2/train/adas/adas_79.jpg
/content/DS2/validation/adas/adas_83.jpg
/content/DS2/train/adas/adas_83.jpg
/content/DS2/validation/adas/adas_63.jpg
/content/DS2/train/adas/adas_63.jpg
/content/DS2/validation/adas/adas_46.jpg
/content/DS2/train/adas/adas_46.jpg
/content/DS2/validation/adas/adas_84.jpg
/content/DS2/train/adas/adas_84.jpg
/content/DS2/validation/adas/adas_31.jpg
/content/DS2/train/adas/adas_31.jpg
/content/DS2/validation/adas/adas_26.jpg
/content/DS2/train/adas/adas_26.jpg
/content/DS2/validation/adas/adas_35.jpg
/content/DS2/train/adas/adas_35.jpg
/content/DS2/validation/adas/adas_86.jpg
/content/DS2/train/adas/adas_86.jpg
/content/DS2/validation/adas/adas_62.jpg
/content/DS2/train/adas/adas_62.jpg
/content/DS2/validation/adas/adas_34.jpg
/content/DS2/train/adas/adas_34.jpg

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
4/4 [==============================] - 69s 21s/step - loss: 2.4066 - accuracy: 0.1226
Epoch 2/10
4/4 [==============================] - 67s 15s/step - loss: 2.2061 - accuracy: 0.1352
Epoch 3/10
4/4 [==============================] - 67s 15s/step - loss: 2.1107 - accuracy: 0.2107
Epoch 4/10
4/4 [==============================] - 69s 16s/step - loss: 2.0264 - accuracy: 0.2893
Epoch 5/10
4/4 [==============================] - 69s 16s/step - loss: 1.9782 - accuracy: 0.2893
Epoch 6/10
4/4 [==============================] - 67s 15s/step - loss: 1.8927 - accuracy: 0.3208
Epoch 7/10
4/4 [==============================] - 65s 15s/step - loss: 1.8031 - accuracy: 0.3931
Epoch 8/10
4/4 [==============================] - 65s 15s/step - loss: 1.7863 - accuracy: 0.3899
Epoch 9/10
4/4 [==============================] - 66s 15s/step - loss: 1.7200 - accuracy: 0.4245
Epoch 10/10
4/4 [==============================] - 67s 15s/step - loss: 1.6694 - accuracy: 0.5000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


2/2 [==============================] - 32s 12s/step
***Performance on Validation data***
Accuracy  : 0.5534591194968553
Precision : 0.5346029232821685
f1Score : 0.5157280646616167
[[12  0  8  0  0  0  0  0  0  0]
 [ 0 16  0  1  0  0  0  1  0  0]
 [ 4  0 16  0  0  0  0  0  0  0]
 [ 0  1  0  6  0  1  0  6  0  0]
 [ 3  0  6  0  1  0  0  0  4  0]
 [ 0  1  0  1  0  9  0  2  0  0]
 [ 0  1  0  1  0  0  8  4  0  0]
 [ 0  4  0  6  0  1  0  9  0  0]
 [ 1  0  0  0  1  0  0  0 11  0]
 [ 2  4  0  3  0  0  0  4  0  0]]
/content/DS2/validation/adas/adas_72.jpg
/content/DS2/train/adas/adas_72.jpg
/content/DS2/validation/adas/adas_80.jpg
/content/DS2/train/adas/adas_80.jpg
/content/DS2/validation/adas/adas_59.jpg
/content/DS2/train/adas/adas_59.jpg
/content/DS2/validation/adas/adas_34.jpg
/content/DS2/train/adas/adas_34.jpg
/content/DS2/validation/adas/adas_85.jpg
/content/DS2/train/adas/adas_85.jpg
/content/DS2/validation/adas/adas_31.jpg
/content/DS2/train/adas/adas_31.jpg
/content/DS2/validation/ada

...

2/2 [==============================] - 32s 12s/step
***Performance on Validation data***
Accuracy  : 0.7987421383647799
Precision : 0.8517781512319745
f1Score : 0.7829822843485824
[[14  0  6  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  1  1  0  0  0]
 [ 1  0 18  0  1  0  0  0  0  0]
 [ 0  1  0  7  0  0  0  6  0  0]
 [ 0  0  0  0 12  0  0  0  1  0]
 [ 0  1  0  0  0 12  0  1  0  0]
 [ 0  0  0  0  0  0 12  1  0  0]
 [ 0  0  0  0  0  0  0 20  0  0]
 [ 0  0  0  0  0  0  0  0 14  0]
 [ 0  0  0  1  0  0  0 10  0  2]]
/content/DS2/validation/adas/adas_78.jpg
/content/DS2/train/adas/adas_78.jpg
/content/DS2/validation/adas/adas_64.jpg
/content/DS2/train/adas/adas_64.jpg
/content/DS2/validation/adas/adas_76.jpg
/content/DS2/train/adas/adas_76.jpg
/content/DS2/validation/adas/adas_26.jpg
/content/DS2/train/adas/adas_26.jpg
/content/DS2/validation/adas/adas_32.jpg
/content/DS2/train/adas/adas_32.jpg
/content/DS2/validation/adas/adas_73.jpg
/content/DS2/train/adas/adas_73.jpg
/content/DS2/validation/ada

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
4/4 [==============================] - 68s 15s/step - loss: 1.3246 - accuracy: 0.6384
Epoch 2/10
4/4 [==============================] - 66s 15s/step - loss: 1.2696 - accuracy: 0.6824
Epoch 3/10
4/4 [==============================] - 67s 15s/step - loss: 1.2480 - accuracy: 0.7075
Epoch 4/10
4/4 [==============================] - 66s 15s/step - loss: 1.2234 - accuracy: 0.7013
Epoch 5/10
4/4 [==============================] - 66s 21s/step - loss: 1.2230 - accuracy: 0.6730
Epoch 6/10
4/4 [==============================] - 67s 15s/step - loss: 1.2264 - accuracy: 0.6352
Epoch 7/10
4/4 [==============================] - 65s 15s/step - loss: 1.1634 - accuracy: 0.7138
Epoch 8/10
4/4 [==============================] - 66s 15s/step - loss: 1.1306 - accuracy: 0.7390
Epoch 9/10
4/4 [==============================] - 68s 21s/step - loss: 1.1239 - accuracy: 0.7264
Epoch 10/10
4/4 [==============================] - 68s 16s/step - loss: 1.0981 - accuracy: 0.7358


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


2/2 [==============================] - 32s 12s/step
***Performance on Validation data***
Accuracy  : 0.7610062893081762
Precision : 0.7542182077284885
f1Score : 0.7295669594828899
[[17  0  3  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0]
 [ 0  5  0  4  0  0  1  2  0  2]
 [ 0  0  0  0  8  0  0  0  5  0]
 [ 0  0  0  0  0 13  0  0  0  0]
 [ 0  0  0  0  0  0 11  3  0  0]
 [ 0  1  0  2  0  0  1 16  0  0]
 [ 0  0  0  0  0  0  0  0 13  0]
 [ 1  7  0  0  0  0  1  4  0  1]]


In [ ]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) 
predictions = model3.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore = my_metrics(true_classes, yPredictions)
model.save(MODEL_FILENAME)

==============TEST RESULTS============
Found 118 images belonging to 10 classes.
2/2 [==============================] - 24s 4s/step
Accuracy  : 0.7372881355932204
Precision : 0.7676565541470826
f1Score : 0.7213264607332404
[[12  0  3  0  0  0  0  0  0  0]
 [ 0 11  0  0  0  1  0  1  0  0]
 [ 0  0 15  0  0  0  0  0  0  0]
 [ 0  3  0  3  0  3  1  0  0  0]
 [ 0  0  0  0  5  0  0  0  5  0]
 [ 0  0  0  0  0  9  0  0  0  1]
 [ 0  0  0  0  0  1  7  2  0  0]
 [ 0  1  0  2  0  0  0 12  0  0]
 [ 0  0  0  0  0  0  0  0 10  0]
 [ 0  5  0  0  0  0  0  2  0  3]]


In [13]:
import pandas as pd
df = pd.DataFrame(
    {
        "Training accuracy": [0.9874213836477987, 0.9937106918238994,0.7610062893081762],
     "Validation accuracy": [0.9830508474576272,0.9830508474576272,0.7372881355932204],
    }, index = ["Variation 1", "Variation 2", "Variation 3"]
)

In [14]:
df

,Training accuracy,Validation accuracy
Variation 1,0.987421,0.983051
Variation 2,0.993711,0.983051
Variation 3,0.761006,0.737288
